In [ ]:
pip install import-ipynb

In [ ]:
pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 76.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Successfully uninstalled spacy-3.7.5


In [ ]:
pip install vk-api

In [ ]:
pip install emoji

In [6]:
from config import TOKEN, VERSION
import import_ipynb
import vk_api
from vk_get_data_functions import create_all_posts_csv
from google.colab import drive
import spacy
import pandas as pd
import emoji

In [ ]:
pip install -U pip setuptools wheel

In [ ]:
drive.mount('/content/gdrive')
gc_folder = r"/content/gdrive/My Drive/sns4human/data/vk"

In [ ]:
!python -m spacy download ru_core_news_sm

In [10]:
nlp = spacy.load("ru_core_news_sm")
domains_groups = ['aparfenchikov', 'minnazrk', 'rk_nationalmuseum', 'olonmus']

In [ ]:
for domain in domains_groups:
     create_all_posts_csv(TOKEN, VERSION, domain)

In [9]:
pass_chars = ['"', '\'', '.', ',', '\n', ':', ';', '?', '!', '', ' ',  '-', '_']
posts = []
tokens = []
unique_words = []
mid_len_post = []
alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя-abcdefghijklmnopqrstuvwxyz')

def lemmatization(sent):
    doc = nlp(sent)
    hashtag_flag = False
    cnt_tokens = 0
    for token in doc:
        cnt_tokens += 1
        if hashtag_flag:
            lemma = '#' + token.lemma_
            hashtag_flag = False
        else:
            if token.text == '#':
                hashtag_flag = True
                continue
            set_word = set(str(token).lower())
            if str(token) in pass_chars or ":" in emoji.demojize(str(token)) or not set_word.issubset(alphabet):
                continue
            lemma = token.lemma_
        if dict_lem.get(lemma, 0) == 0:
            dict_lem[lemma] = 1
        else:
            dict_lem[lemma] += 1
    return cnt_tokens

for domain in domains_groups:
    p = 0
    t = 0
    dict_lem = {}
    df = pd.read_csv(domain + "_all_posts.csv", header=None, usecols=[1])
    for items in df.items():
        for i in range(1, len(items[1])):
            p += 1
            t += lemmatization(str(items[1][i]))
        mid_len_post.append(t // p)
        dict_lem = dict(sorted(dict_lem.items(), key=lambda item: item[1], reverse=True))
        tokens.append(t)
        posts.append(p)
        unique_words.append(len(dict_lem))
d = {'Группа ВК': domains_groups, 'Количество постов': posts, 'Количество токенов': tokens, 'Количество уникальных слов': unique_words, 'Средняя длина поста в словах': mid_len_post}
df = pd.DataFrame(data=d)
df.to_csv('statistics.csv', index=False, encoding='utf-8')

In [ ]:
import pdb
pdb.set_trace()

In [11]:
official_groups = {}
museum_groups = {}
pass_chars = ['"', '\'', '.', ',', '\n', ':', ';', '?', '!','','-','_']
domains_groups = ['aparfenchikov', 'minnazrk', 'rk_nationalmuseum', 'olonmus']
for domain in domains_groups:
    # частотные слова - топ 100 по частотности
    with open(gc_folder + "/" + domain + "_all_posts.csv", encoding='utf-8') as text:
        text.readline()
        for line in text:
            doc = nlp(line)
            for token in doc:
                if token in pass_chars:
                    continue
                lemma = token.lemma_
                if domain == 'aparfenchikov' or domain == 'minnazrk':
                    if official_groups.get(lemma, 0) == 0:
                        official_groups[lemma] = 1
                    else:
                        official_groups[lemma] += 1
                else:
                    if museum_groups.get(lemma, 0) == 0:
                        museum_groups[lemma] = 1
                    else:
                        museum_groups[lemma] += 1
official_groups = dict(sorted(official_groups.items(), key=lambda item: item[1]))
museum_groups = dict(sorted(museum_groups.items(), key=lambda item: item[1]))

official_groups = dict(sorted(official_groups.items(), key=lambda item: item[1], reverse=True))
museum_groups = dict(sorted(museum_groups.items(), key=lambda item: item[1], reverse=True))
freq_words_offical = {}
freq_words_museum = {}
i = 0

for lemma, count in official_groups.items():
    if i == 100:
        break
    if lemma not in museum_groups or museum_groups[lemma] == 1:
        freq_words_offical[lemma] = count
        i += 1

j = 0
for lemma, count in museum_groups.items():
    if j == 100:
        break
    if lemma not in official_groups or official_groups[lemma] == 1:
        freq_words_museum[lemma] = count
        j += 1

d = {'Частотные слова, которых нет в музейных группах': freq_words_offical.keys(),'Частота':freq_words_offical.values()}
df = pd.DataFrame(data=d)
df.to_csv(gc_folder +  '/'+ 'frequency_official_words.csv', index=False, encoding='utf-8')

d = {'Частотные слова, которых нет в официальных группах': freq_words_museum.keys(),'Частота':freq_words_museum.values()}
df = pd.DataFrame(data=d)
df.to_csv(gc_folder +  '/'+'frequency_museum_words.csv', index=False, encoding='utf-8')

TypeError: Cannot convert numpy.ndarray to numpy.ndarray